# Data validation between the data set in BigQuery and GraphDB

The objectives are to:
1. Ensure complete upload of data to BigQuery and GraphDB.
2. Check for data qualtiy issues.
3. Gain insights on the data.
4. Engineer features.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Mount to Google Drive to save results
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MSc/2020-21/Research\ Project/Colab/
%ls

Mounted at /content/drive
/content/drive/MyDrive/MSc/2020-21/Research Project/Colab
label_count_df.csv


In [3]:
# Connect to GCP Bucket
from google.colab import auth
auth.authenticate_user()

In [4]:
# Set GCP project ID and region to Europe West 2 - London
PROJECT = 'detect-fake-news-313201'
!gcloud config set project $PROJECT
REGION = 'europe-west2'
CLUSTER = '{}-cluster'.format(PROJECT)
!gcloud config set compute/region $REGION
!gcloud config set dataproc/region $REGION

!gcloud config list # show some information

Updated property [core/project].
Updated property [compute/region].
Updated property [dataproc/region].
[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
region = europe-west2
[core]
account = aaron.altrock@gmail.com
project = detect-fake-news-313201
[dataproc]
region = europe-west2

Your active configuration is: [default]


## Check the number of files in successive GCP cloud storage buckets

In [5]:
# Count the number of cleaned JSON files from the end of stage 1 in the pipeline
!gsutil ls -l gs://fake_news_cleaned_json/*.json | wc -l

59733


In [6]:
# Count the number of parsed JSON and TTL files into triples at the end of stage 2 in the pipeline
!gsutil ls -l gs://fake_news_ttl_json/*.ttl | wc -l
!gsutil ls -l gs://fake_news_ttl_json/*.json | wc -l

27590
27590


The variance between the 59,733 cleaned files to 27,590 turtle documents would suggest this is due to the raw data containing duplicating records for the same news web page, when the turtles are indexed by the hash value of the URLs and therefore would overwrite leading to small number of samples.

In [7]:
# Based on https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries#bigquery_simple_app_client-python
from google.cloud import bigquery
client = bigquery.Client(PROJECT)


## Profile the data in its original form held in BigQuery

In [8]:
# BIgQuery data row count
query_job = client.query(
    """
    SELECT COUNT(*) AS POPULATION_COUNT
    FROM `detect-fake-news-313201.fake_news_sql.src_fake_news`
    """
)

res_df = query_job.result().to_dataframe()  # Waits for job to complete.

res_df

,POPULATION_COUNT
0,27589


Therefore deviation by one record compared to the number of files in `gs://fake_news_ttl_json`.

In [9]:
# BIgQuery data row count
query_job = client.query(
    """
    WITH URL_LIST AS (
      SELECT 
      URL
      , COUNT(*) AS URL_COUNT
      FROM `detect-fake-news-313201.fake_news_sql.src_fake_news`
      GROUP BY URL
    )
    SELECT * FROM URL_LIST WHERE URL_COUNT > 1
    """
)

res_df = query_job.result().to_dataframe()  # Waits for job to complete.

res_df

,URL,URL_COUNT


Therefore no samples found to have duplicating URL in the BigQuery table, and all articles have unique URLs.

In [10]:
# BIgQuery data preview
query_job = client.query(
    """
    SELECT *
    FROM `detect-fake-news-313201.fake_news_sql.src_fake_news`
    LIMIT 10
    """
)

res_df = query_job.result().to_dataframe()  # Waits for job to complete.

res_df

,url,domain,domain_hash,title_hash,title,body_hash,body,label,url_hash
0,http://awm.com/woman-adopts-rescue-dog-starts-...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_761e82d05283bff6f1832c7164626e26,"Woman Adopts Rescue Dog, Starts Noticing Foul ...",body_072ebc1af96077202eb2ad843b6fe4cd,Kelly Benzel was looking for a new best friend...,unreliable,00d76dc2a2b9c02526ab6aba99cea68d
1,http://awm.com/grand-daughter-sings-favorite-s...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_2a30a9b6a5a75250a7317c60c0f871e6,Grand Daughter Sings Favorite Song to Grandma ...,body_3e0c1daeb28fa250067a74240a5af93d,Dementia is one of the scariest and heartbreak...,unreliable,022b655e21ca0d38a1fc33f2a37165a8
2,http://awm.com/first-class-passenger-sees-a-so...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_b89b152a0c80235c4e6174a96e98f166,First Class Passenger Sees A Soldier Walking U...,body_6953f3cbe02f2bdf2083937b1c8f9b7f,Air travel can be a tiring and trying experien...,unreliable,0405cdbffb7b0f0aef94eafaba0863ff
3,http://awm.com/model-who-boasts-she-transforme...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_9868eac5482fdbfebf7b94494bd078ec,Model Who Boasts She Transformed Into An Afric...,body_d9e3185ae65d5b97185f08d85c31abec,"Martina Big, a 28-year-old white German model,...",unreliable,09f269f0d217228587029f2012f875c9
4,http://awm.com/dad-puts-his-son-into-the-game-...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_08db9d62bccf3a51cb7ded0fdf666568,Dad Puts His Son Into The Game Days After Beat...,body_fcb8636125a87f14428856aff72e307f,A cancer diagnosis never comes at the right ti...,unreliable,0c739d0432f31e7805c0d62f6eff0849
5,http://awm.com/little-girls-reaction-to-shooti...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_0ef200f6649ab2907625d4db1f796c87,Little Girl’s Reaction To Shooting Her First D...,body_c8d4436c055c1516d67af592b850a17f,"In the YouTube video below, you’re going to se...",unreliable,154032234b0955080af9117e817a7676
6,http://awm.com/out-of-all-the-holiday-recipes-...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_0165dfcd088b4df972d8b23ee45198e0,"Out Of All The Holiday Recipes I’ve Tried, His...",body_7a5cb6d058bda9afc409df365438cc7f,"Once a year, the grocery stores start stocking...",unreliable,1af9a9f5326071c7bc5a6e16e04a75df
7,http://awm.com/a-starbucks-barista-couldnt-fin...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_55c16d853bfc26cc4697fe2158f2e273,A Starbucks Barista Couldn’t Find A Babysitter...,body_b2277eca06c8bd84aba7a6512dcbb5e6,Starbucks can be a great place to get luxuriou...,unreliable,201402629241e8945bc127ac0b387dc8
8,http://awm.com/the-easily-offended-whine-when-...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_ab7f6ea47a85aeacab878345864501df,The Easily Offended Whine When Kelly Clarkson ...,body_ddf7af702f3aef83dcb93cc5717ad34f,Dolling out corporal punishment is a controver...,unreliable,21825e99b66d5a6b55e9e3f8c5bdfe71
9,http://awm.com/during-heated-love-making-wife-...,awm.com,domain_9725d0802cf38288710d8bff8f64dcba,title_3ed2e9ed1879767b09e706f2a7f5ced5,"During Heated Love Making, Wife Accidentally B...",body_f62f488c90df847302cd010ff91e016b,He always told her that he liked it rough. But...,unreliable,24c1a494a889e14ef274e14d31fca550


In [11]:
# BIgQuery count by domain
query_job = client.query(
    """
    SELECT
    DOMAIN_HASH
    , LABEL
    , COUNT(*) AS ARTICLES_COUNT
    FROM `detect-fake-news-313201.fake_news_sql.src_fake_news`
    GROUP BY DOMAIN_HASH, LABEL
    ORDER BY ARTICLES_COUNT DESC
    """
)

res_df = query_job.result().to_dataframe()  # Waits for job to complete.

# Tally the domain hash to ensure each domain only has one label
domain_tally_ls = []
duplicate_domain_tally_ls = []
for i, row in res_df.iterrows():
  if row['DOMAIN_HASH'] in domain_tally_ls:
    duplicate_domain_tally_ls += [True]
  else:
    duplicate_domain_tally_ls += [False]
  
  # Add domain hash to the list of domains already reviwed
  domain_tally_ls += [row['DOMAIN_HASH']]

res_df['DOMAIN_HAS_MULTIPLE_LABEL'] = duplicate_domain_tally_ls

res_df

,DOMAIN_HASH,LABEL,ARTICLES_COUNT,DOMAIN_HAS_MULTIPLE_LABEL
0,domain_8f00b2b61ba335244231d632d390bf8d,fake,7046,False
1,domain_9f5bbbcbad4a48edd86162fabbe90b6e,political,5441,False
2,domain_7e058b16a00bca2c3620d5147881e34d,conspiracy,2819,False
3,domain_98845529bda170844508657ae469197a,bias,1162,False
4,domain_2bb13bea21c458421e33179105662bdd,political,1061,False
5,domain_bd8db3098e8505e2d80c8b89a2eccdb8,clickbait,813,False
6,domain_0d6ca907eef322628f81ae5a38735d44,bias,730,False
7,domain_f573aed8d0dcf08ef6ce3efd5f5c48c0,political,729,False
8,domain_213f2e7f6968dd534eb43ef113fce3e1,political,671,False
9,domain_8fbb80fdc3a337fd46babecb01f203fa,junksci,615,False


In [12]:
# BIgQuery count by label
query_job = client.query(
    """
    SELECT
    LABEL
    , COUNT(*) AS LABEL_COUNT
    FROM `detect-fake-news-313201.fake_news_sql.src_fake_news`
    GROUP BY LABEL
    ORDER BY LABEL_COUNT DESC
    """
)

res_df = query_job.result().to_dataframe()

print('Total: {}'.format(res_df['LABEL_COUNT'].sum()))

res_df

Total: 27589


,LABEL,LABEL_COUNT
0,political,9776
1,fake,7081
2,conspiracy,3512
3,bias,2793
4,clickbait,1238
5,junksci,993
6,None,990
7,unreliable,437
8,unknown,354
9,reliable,178


In [13]:
# BIgQuery list of URLs
query_job = client.query(
    """
    SELECT DISTINCT
    URL_HASH
    FROM `detect-fake-news-313201.fake_news_sql.src_fake_news`
    """
)

res_df = query_job.result().to_dataframe()

print('Total: {}'.format(res_df.shape[0]))

res_df.head()

Total: 27589


,URL_HASH
0,00d76dc2a2b9c02526ab6aba99cea68d
1,022b655e21ca0d38a1fc33f2a37165a8
2,0405cdbffb7b0f0aef94eafaba0863ff
3,09f269f0d217228587029f2012f875c9
4,0c739d0432f31e7805c0d62f6eff0849


Noted that there were no classification for 990 samples, and further 354 with unknown classifications.



## Profile the data in GraphDB

In [14]:
# Install the wrapper package
# Source: https://github.com/RDFLib/sparqlwrapper
!pip install sparqlwrapper

     |████████████████████████████████| 407 kB 8.0 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 


In [15]:
# Code based on: https://sparqlwrapper.readthedocs.io/en/latest/main.html
from SPARQLWrapper import SPARQLWrapper, JSON

queryString = """
PREFIX aa: <http://www.city.ac.uk/ds/inm363/aaron_altrock#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select (count(?url_hash) as ?url_count) where {
  ?url_hash rdf:type aa:urlHash .
}
"""


sparql = SPARQLWrapper("http://35.246.120.165:7200/repositories/src_fake_news")
sparql.setReturnFormat(JSON)
sparql.setQuery(queryString)

try :
   res_dct = sparql.query().convert()
   print('OK')

except Exception as e:
   print('ERROR: {}'.format(e))
   

OK


In [16]:
# No. of URL hash in GraphDB
res_dct.get('results').get('bindings')[0].get('url_count').get('value')

'27589'

Therefore noted that the number of news articles as URL hashes were completely uploaded when compared to BigQury count given both have the same number of articles `27598`.

In [17]:
res_dct

{'head': {'vars': ['url_count']},
 'results': {'bindings': [{'url_count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '27589'}}]}}

### No. of URL hashes

In [18]:
# Code based on: https://sparqlwrapper.readthedocs.io/en/latest/main.html

queryString = """
PREFIX aa: <http://www.city.ac.uk/ds/inm363/aaron_altrock#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select ?domain_hash ?label (count(?url_hash) as ?url_count) where {
  ?domain_hash rdf:type aa:domainHash .
  ?url_hash rdf:type aa:urlHash .
  ?label rdf:type aa:newsLabel .
  ?url_hash aa:has_domain_hash ?domain_hash .
  ?url_hash aa:has_news_label ?label .
}
GROUP BY ?domain_hash ?label
ORDER BY ?url_count
"""


sparql = SPARQLWrapper("http://35.246.120.165:7200/repositories/src_fake_news")
sparql.setReturnFormat(JSON)
sparql.setQuery(queryString)

try :
   res_dct = sparql.query().convert()
   print('OK')

except Exception as e:
   print('ERROR: {}'.format(e))
   

OK


In [19]:
res_dct

{'head': {'vars': ['domain_hash', 'label', 'url_count']},
 'results': {'bindings': [{'domain_hash': {'type': 'uri',
     'value': 'http://www.city.ac.uk/ds/inm363/aaron_altrock#domain_02cb0e25d8e2a8950c674f2d77e05712'},
    'label': {'type': 'uri',
     'value': 'http://www.city.ac.uk/ds/inm363/aaron_altrock#political'},
    'url_count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '1'}},
   {'domain_hash': {'type': 'uri',
     'value': 'http://www.city.ac.uk/ds/inm363/aaron_altrock#domain_06d7d51d4c0251a3b42ebbfe7a26f5b4'},
    'label': {'type': 'uri',
     'value': 'http://www.city.ac.uk/ds/inm363/aaron_altrock#bias'},
    'url_count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '1'}},
   {'domain_hash': {'type': 'uri',
     'value': 'http://www.city.ac.uk/ds/inm363/aaron_altrock#domain_1a482c97f790497c9809e63070e977dc'},
    'label': {'type': 'uri',
     'value': 'http://www.city.ac.

In [20]:
import re
res_ls = res_dct.get('results').get('bindings')

# Helper func to transform SPARQLWrapper query result output (dict) to Pandas dataframe
def parse_to_dataframe(res_ls):

  # If query result has content then parse to data frame else return None
  if len(res_ls) > 0:
    # Get column names
    col_nm_ls = list(res_ls[0].keys())

    parsed_res_ls = []

    for res_dct in res_ls:
      __res_ls = []
      for k, v in res_dct.items():
        __res_ls += [v.get('value')]

      # __res_ls = [res_dct]
      parsed_res_ls += [__res_ls]
    
    res_df = pd.DataFrame.from_dict(parsed_res_ls)
    res_df.columns = col_nm_ls
    res_df.reset_index(inplace=True, drop=True)

    return res_df

  else:
    return None

# Parse dict output from SPARQL to Pandas data frame
domain_url_count_df = parse_to_dataframe(res_ls)

# Remove name space prefix
domain_url_count_df['domain_hash'] = domain_url_count_df['domain_hash'].map(lambda str: str[str.find('#') + 1:])
domain_url_count_df['label'] = domain_url_count_df['label'].map(lambda str: str[str.find('#') + 1:])

# Convert url_count to integer
domain_url_count_df['url_count'] = domain_url_count_df['url_count'].map(int)

# Count percentage
domain_url_count_df['url_count_pct'] = domain_url_count_df['url_count'].map(lambda val: val / domain_url_count_df['url_count'].sum() * 100)

domain_url_count_df.sort_values(by=['url_count'], ascending=False, inplace=True)

domain_url_count_df

,domain_hash,label,url_count,url_count_pct
202,domain_8f00b2b61ba335244231d632d390bf8d,fake,7046,25.539164
201,domain_9f5bbbcbad4a48edd86162fabbe90b6e,political,5441,19.721628
200,domain_7e058b16a00bca2c3620d5147881e34d,conspiracy,2819,10.217840
199,domain_98845529bda170844508657ae469197a,bias,1162,4.211824
198,domain_2bb13bea21c458421e33179105662bdd,political,1061,3.845736
197,domain_bd8db3098e8505e2d80c8b89a2eccdb8,clickbait,813,2.946827
196,domain_0d6ca907eef322628f81ae5a38735d44,bias,730,2.645982
195,domain_f573aed8d0dcf08ef6ce3efd5f5c48c0,political,729,2.642357
194,domain_213f2e7f6968dd534eb43ef113fce3e1,political,671,2.432129
193,domain_8fbb80fdc3a337fd46babecb01f203fa,junksci,615,2.229149


In [21]:
# Summary of the classification by count of articles
label_count_df = domain_url_count_df[['label', 'url_count']].groupby('label').sum().sort_values(by='url_count', ascending=False).reset_index(drop=False)
total_rec_count = sum(label_count_df['url_count'].to_list())
print('Total number of articles: {}'.format(total_rec_count))
label_count_df['url_count_pct'] = label_count_df['url_count'].map(lambda val: val / total_rec_count * 100 if val is not None else 0)
label_count_df

Total number of articles: 27589


,label,url_count,url_count_pct
0,political,9776,35.434412
1,fake,7081,25.666026
2,conspiracy,3512,12.729711
3,bias,2793,10.123600
4,clickbait,1238,4.487296
5,junksci,993,3.599261
6,,990,3.588387
7,unreliable,437,1.583965
8,unknown,354,1.283120
9,reliable,178,0.645185


Per the https://github.com/several27/FakeNewsCorpus details on each news article classification noted no such classifications as `unknown` nor a classification of `None`.  These may be added by the author of the data set who performed scrapping to backfill classifications that could not be found.

In [22]:
# Save summary to CSV
label_count_df.to_csv(r'label_count_df.csv', index=False)

## No. of domains by classification

In [27]:
print('No. of domains in total: {}'.format(domain_url_count_df['domain_hash'].shape[0]))
domain_count_df = domain_url_count_df[['domain_hash', 'label']].groupby(by='label').count().sort_values(by='domain_hash', ascending=False).reset_index()
domain_count_df.rename(columns={'domain_hash': 'domain_count'}, inplace=True)
domain_count_df['domain_count_pct'] = domain_count_df['domain_count'].map(lambda val: val / domain_count_df['domain_count'].sum() * 100)
domain_count_df.to_csv('domain_count_df.csv', index=False)
domain_count_df

No. of domains in total: 203


,label,domain_count,domain_count_pct
0,political,40,19.704433
1,bias,37,18.226601
2,conspiracy,30,14.778325
3,,19,9.359606
4,unknown,15,7.389163
5,unreliable,13,6.403941
6,clickbait,11,5.418719
7,junksci,11,5.418719
8,satire,10,4.926108
9,fake,9,4.433498


## Repeated publication of the same news articles

In [35]:
# Code based on: https://sparqlwrapper.readthedocs.io/en/latest/main.html

queryString = """
PREFIX aa: <http://www.city.ac.uk/ds/inm363/aaron_altrock#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select ?domain_hash ?url_hash ?body_hash ?label where {
  ?domain_hash rdf:type aa:domainHash .
  ?url_hash rdf:type aa:urlHash .
  ?body_hash rdf:type aa:bodyHash .
  ?label rdf:type aa:newsLabel .
  ?url_hash aa:has_domain_hash ?domain_hash .
  ?url_hash aa:has_news_label ?label .
  ?url_hash aa:has_body_hash ?body_hash .
}
ORDER BY ?url_count
"""


sparql = SPARQLWrapper("http://35.246.120.165:7200/repositories/src_fake_news")
sparql.setReturnFormat(JSON)
sparql.setQuery(queryString)

try :
   res_dct = sparql.query().convert()
   print('OK')

except Exception as e:
   print('ERROR: {}'.format(e))


# Parse dict output from SPARQL to Pandas data frame
res_ls = res_dct.get('results').get('bindings')
domain_url_body_df = parse_to_dataframe(res_ls)

# Remove name space prefix
domain_url_body_df['domain_hash'] = domain_url_body_df['domain_hash'].map(lambda str: str[str.find('#') + 1:])
domain_url_body_df['url_hash'] = domain_url_body_df['url_hash'].map(lambda str: str[str.find('#') + 1:])
domain_url_body_df['body_hash'] = domain_url_body_df['body_hash'].map(lambda str: str[str.find('#') + 1:])
domain_url_body_df['label'] = domain_url_body_df['label'].map(lambda str: str[str.find('#') + 1:])

domain_url_body_df.head()

OK


,domain_hash,url_hash,body_hash,label
0,domain_8f00b2b61ba335244231d632d390bf8d,e681402ef05d57310574314a7918aa6c,body_0002ffe45350c699360f71e143099f38,fake
1,domain_8f00b2b61ba335244231d632d390bf8d,3e3b0220abefd45f00916f2b69d13051,body_0004e310490d5711e38e13a0b0d6572b,fake
2,domain_8f00b2b61ba335244231d632d390bf8d,7c6de65487ed9706801ee482e5f4343f,body_000767e358f2bd1225e8d3fe59a65276,fake
3,domain_3e2b4bc8d67cf01a6f156413acc03375,4a883a995ada072079326ffb6d55f992,body_0008c819ef9503a60019f5b971b972e3,clickbait
4,domain_9c34eb473390528f31c0303bda283c21,a4d57e74eaf6511fdcb675e1c7b55a15,body_0009bb573f65b94ab468528099f568cf,unreliable


In [40]:
domain_url_body_df[['body_hash', 'label', 'url_hash']].groupby(['body_hash', 'label']).count().reset_index().sort_values(by='url_hash', ascending=False).head(50)

,body_hash,label,url_hash
15548,body_a91766cb7db645f57aa52fca4b5a77e0,bias,410
12169,body_845af446af521c88c801695145743815,junksci,325
11210,body_7a03935701b11bf99ae50445a0e67793,fake,82
16030,body_ae1d6ee9a8d8345cd74e6983178a7a45,political,52
20048,body_d9ad78c5666f8bcb7c2b954427cdaa2b,political,43
3482,body_24dc2baef88f46a4c00982d270aa8388,bias,33
8956,body_61a2d8ac5c204e2b39450edef84b3bfa,unknown,24
7585,body_52ea6262c39b053fef53859b11c22898,political,22
17528,body_bdcab4fe046652ff22452f03c706d457,political,20
22566,body_f52d68dc3ff66be08eb737409be420de,political,19
